In [1]:
%pip install pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install opencv-python


In [3]:

import numpy as np 
import pandas as pd 


import os

import cv2

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from tensorflow import keras
from keras.models import Model , load_model
from keras.applications.vgg16 import VGG16
from keras.layers import  Dense, Flatten
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [6]:
IMAGE_SIZE = [224, 224]

path_train = 'Chest X-Ray Images\chest_xray\chest_xray\train'
path_validation = 'Chest X-Ray Images\chest_xray\chest_xray\val'

In [7]:
vgg_model = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [8]:
for layer in vgg_model.layers:
    layer.trainable = False

In [9]:
folders = glob('../input/chest-xray-pneumonia/chest_xray/chest_xray/train/*')
x = Flatten()(vgg_model.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg_model.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_data = train_datagen.flow_from_directory('Chest X-Ray Images\\chest_xray\\chest_xray\\train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
testing_data = test_datagen.flow_from_directory('Chest X-Ray Images\\chest_xray\\chest_xray\\test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [13]:
history = model.fit_generator(
  training_data,
  validation_data=testing_data,
  epochs=1,
  steps_per_epoch=len(training_data),
  validation_steps=len(testing_data),
)

: 

In [ ]:
model.save('chest_xray.h5')

In [ ]:
model=load_model('chest_xray.h5')

In [ ]:
img=image.load_img('Chest X-Ray Images\chest_xray\chest_xray\val\NORMAL\NORMAL2-IM-1430-0001.jpeg')
resized_img = cv2.resize(np.array(img), (224, 224))

In [ ]:
i = image.img_to_array(resized_img)

In [ ]:
i = np.expand_dims(i, axis=0)

In [ ]:
img_data = preprocess_input(i)

In [ ]:
classes = model.predict(img_data)

In [ ]:
result=int(classes[0][0])

In [ ]:
if result == 0:
    print("Person is Affected By PNEUMONIA")
else:
    print("Result is Normal")